<a href="https://colab.research.google.com/github/DataJenius/GPT2_perplexity_repetition/blob/main/2022_02_04__experiment__GPT2_perplexity_repetition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#----------------------------------------
# install requirements - 🤗 Transformers
#----------------------------------------
!pip install -q git+https://github.com/huggingface/transformers.git
!pip install -q tensorflow==2.1
!pip install tensorflow-gpu

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-gpu 2.8.0 requires tensorboard<2.9,>=2.8, but you have tensorboard 2.1.1 which is incompatible.
  Using cached tensorboard-2.8.0-py3-none-any.whl (5.8 MB)
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.1.1
    Uninstalling tensorboard-2.1.1:
      Successfully uninstalled tensorboard-2.1.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.1.0 requires tensorboard<2.2.0,>=2.1.0, but you have tensorboard 2.8.0 which is incompatible.


In [3]:
#------------------------
# load all dependencies
#------------------------
import tensorflow as tf
from transformers import GPT2LMHeadModel, TFGPT2LMHeadModel, GPT2Tokenizer
import torch
from tqdm import tqdm
import nltk
from nltk.collocations import *
from nltk.tokenize import RegexpTokenizer
import string
import pandas as pd
from google.colab import files

ModuleNotFoundError: ignored

In [4]:
#-------------------------------------
# load models and tokenizer from 🤗
#-------------------------------------
# using GPT2-large
device = "cuda"
model_id = "gpt2-large"

# load our tokenizer
tokenizer = GPT2Tokenizer.from_pretrained(model_id)

# using two models because I can't find a perplexity calculation with TFGPT2LMHeadModel
# load models with the EOS token as PAD token to avoid warnings
gmodel = TFGPT2LMHeadModel.from_pretrained(model_id, pad_token_id=tokenizer.eos_token_id)
emodel = GPT2LMHeadModel.from_pretrained(model_id, pad_token_id=tokenizer.eos_token_id).to(device)

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/666 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.88G [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at gpt2-large.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


Downloading:   0%|          | 0.00/3.02G [00:00<?, ?B/s]

In [5]:
#----------------------------------------
# Load human text & prompts from Github
#----------------------------------------
# load into Pandas df
human_text = pd.read_csv("https://raw.githubusercontent.com/DataJenius/GPT2_perplexity_repetition/main/data_human_text_and_prompts.csv")

# sanity check it loaded correctly...
#print(human_text.iloc[0,0]) # Source
#print(human_text.iloc[0,4]) # Prompt

In [54]:
#------------------
# Beam Search
#------------------
# deterministic; no need to set random seed

# hold output in a dataframe (easily saved/shared as csv)
output_df = pd.DataFrame()

# iterate through all of our prompts based on human text
for i, row in human_text.iterrows():
  print(human_text.iloc[i,0])
  print(human_text.iloc[i,4])  

  # tokenize our text promp
  input_ids = tokenizer.encode(human_text.iloc[i,4], return_tensors='tf')

  # set model parameters
  # generate 10 outputs of max length 250
  beam_outputs = gmodel.generate(
      input_ids, 
      num_return_sequences=10,    
      max_length=250, 
      num_beams=15, 
      early_stopping=True)

  # hold output in a dataframe (easily saved/shared as csv)
  for j, beam_output in enumerate(beam_outputs):
    output_text = "{}".format(tokenizer.decode(beam_output, skip_special_tokens=True))  
    output_df = output_df.append(pd.DataFrame({'method':['Beam Search'],
                                              'source':[human_text.iloc[i,0]],
                                              'prompt':[human_text.iloc[i,4]],
                                              'trial':[j+1],
                                              'generated_text':[output_text]}))

# save our results to local CSV after iterating through all prompts
output_df.to_csv('data_generated_text_beam_search.csv') 
files.download('data_generated_text_beam_search.csv')

Tale of Two Cities
It was the best of times, it was the worst of times, it was 


ResourceExhaustedError: ignored

In [ ]:
#------------------
# Sampling
#------------------
# set seed to reproduce results
tf.random.set_seed(42)

# hold output in a dataframe (easily saved/shared as csv)
output_df = pd.DataFrame()

# iterate through all of our prompts based on human text
for i, row in human_text.iterrows():
  print(human_text.iloc[i,0])
  print(human_text.iloc[i,4])  

  # tokenize our text promp
  input_ids = tokenizer.encode(human_text.iloc[i,4], return_tensors='tf')

  # set model parameters
  # generate 10 outputs of max length 250
  sampling_outputs = gmodel.generate(    
      input_ids, 
      num_return_sequences=10,    
      max_length=250, 
      do_sample=True,
      top_k=0)

  # hold output in a dataframe (easily saved/shared as csv)
  for j, sampling_output in enumerate(sampling_outputs):
    output_text = "{}".format(tokenizer.decode(sampling_output, skip_special_tokens=True))  
    output_df = output_df.append(pd.DataFrame({'method':['Sampling'],
                                              'source':[human_text.iloc[i,0]],
                                              'prompt':[human_text.iloc[i,4]],
                                              'trial':[j+1],
                                              'generated_text':[output_text]}))

# save our results to local CSV after iterating through all prompts
output_df.to_csv('data_generated_text_sampling.csv') 
files.download('data_generated_text_sampling.csv')

In [ ]:
#--------------------
# Temperature (0.7)
#--------------------
# set seed to reproduce results
tf.random.set_seed(42)

# hold output in a dataframe (easily saved/shared as csv)
output_df = pd.DataFrame()

# iterate through all of our prompts based on human text
for i, row in human_text.iterrows():
  print(human_text.iloc[i,0])
  print(human_text.iloc[i,4])  

  # tokenize our text promp
  input_ids = tokenizer.encode(human_text.iloc[i,4], return_tensors='tf')

  # set model parameters
  # generate 10 outputs of max length 250
  temperature_outputs = gmodel.generate(    
      input_ids, 
      num_return_sequences=10,    
      max_length=250, 
      do_sample=True,
      top_k=0,
      temperature=0.7)

  # hold output in a dataframe (easily saved/shared as csv)
  for j, temperature_output in enumerate(temperature_outputs):
    output_text = "{}".format(tokenizer.decode(temperature_output, skip_special_tokens=True))  
    output_df = output_df.append(pd.DataFrame({'method':['Temperature'],
                                              'source':[human_text.iloc[i,0]],
                                              'prompt':[human_text.iloc[i,4]],
                                              'trial':[j+1],
                                              'generated_text':[output_text]}))

# save our results to local CSV after iterating through all prompts
output_df.to_csv('data_generated_text_temperature.csv') 
files.download('data_generated_text_temperature.csv')

In [ ]:
#--------------------
# Top-K (k=10)
#--------------------
# set seed to reproduce results
tf.random.set_seed(42)

# hold output in a dataframe (easily saved/shared as csv)
output_df = pd.DataFrame()

# iterate through all of our prompts based on human text
for i, row in human_text.iterrows():
  print(human_text.iloc[i,0])
  print(human_text.iloc[i,4])  

  # tokenize our text promp
  input_ids = tokenizer.encode(human_text.iloc[i,4], return_tensors='tf')

  # set model parameters
  # generate 10 outputs of max length 250
  top_k_outputs = gmodel.generate(    
      input_ids, 
      num_return_sequences=10,    
      max_length=250, 
      do_sample=True,
      top_k=10)

  # hold output in a dataframe (easily saved/shared as csv)
  for j, top_k_output in enumerate(top_k_outputs):
    output_text = "{}".format(tokenizer.decode(top_k_output, skip_special_tokens=True))  
    output_df = output_df.append(pd.DataFrame({'method':['Top-K'],
                                              'source':[human_text.iloc[i,0]],
                                              'prompt':[human_text.iloc[i,4]],
                                              'trial':[j+1],
                                              'generated_text':[output_text]}))

# save our results to local CSV after iterating through all prompts
output_df.to_csv('data_generated_text_top_k.csv') 
files.download('data_generated_text_top_k.csv')

In [ ]:
#--------------------
# Top-P (p=0.95)
#--------------------
# set seed to reproduce results
tf.random.set_seed(42)

# hold output in a dataframe (easily saved/shared as csv)
output_df = pd.DataFrame()

# iterate through all of our prompts based on human text
for i, row in human_text.iterrows():
  print(human_text.iloc[i,0])
  print(human_text.iloc[i,4])  

  # tokenize our text promp
  input_ids = tokenizer.encode(human_text.iloc[i,4], return_tensors='tf')

  # set model parameters
  # generate 10 outputs of max length 250
  top_p_outputs = gmodel.generate(    
      input_ids, 
      num_return_sequences=10,    
      max_length=250, 
      do_sample=True,
      top_p=0.95, 
      top_k=0)

  # hold output in a dataframe (easily saved/shared as csv)
  for j, top_p_output in enumerate(top_p_outputs):
    output_text = "{}".format(tokenizer.decode(top_p_output, skip_special_tokens=True))  
    output_df = output_df.append(pd.DataFrame({'method':['Top-P'],
                                              'source':[human_text.iloc[i,0]],
                                              'prompt':[human_text.iloc[i,4]],
                                              'trial':[j+1],
                                              'generated_text':[output_text]}))

# save our results to local CSV after iterating through all prompts
output_df.to_csv('data_generated_text_top_p.csv') 
files.download('data_generated_text_top_p.csv')

In [5]:
#----------------------------------
# Load generated text from Github
#----------------------------------
# load into Pandas dfs
beam_text = pd.read_csv("https://raw.githubusercontent.com/DataJenius/GPT2_perplexity_repetition/main/data_generated_text_beam_search.csv")
sampling_text = pd.read_csv("https://raw.githubusercontent.com/DataJenius/GPT2_perplexity_repetition/main/data_generated_text_sampling.csv")
temperature_text = pd.read_csv("https://raw.githubusercontent.com/DataJenius/GPT2_perplexity_repetition/main/data_generated_text_temperature.csv")
top_k_text = pd.read_csv("https://raw.githubusercontent.com/DataJenius/GPT2_perplexity_repetition/main/data_generated_text_top_k.csv")
top_p_text = pd.read_csv("https://raw.githubusercontent.com/DataJenius/GPT2_perplexity_repetition/main/data_generated_text_top_p.csv")

# sanity check it loaded correctly...
print(beam_text.iloc[0,1]) # Method
print(sampling_text.iloc[0,1]) # Method
print(temperature_text.iloc[0,1]) # Method
print(top_k_text.iloc[0,1]) # Method
print(top_p_text.iloc[0,1]) # Method

# combine into a single dataframe
generated_text_df = beam_text.append(sampling_text).append(temperature_text).append(top_k_text).append(top_p_text)

In [23]:
#---------------------------------------
# Function to calculate Repetitions
# and other basic stats
#---------------------------------------
def calculate_repetitions(my_text, debug=False):
  # ignore capitalization and punctuation
  clean_text = my_text.lower().translate(str.maketrans('', '', string.punctuation))
  character_count = len(my_text)

  # word-based tokens (use spaces)
  tokenizer = nltk.RegexpTokenizer(r"\w+")
  tokens = tokenizer.tokenize(clean_text)
  word_count = len(tokens)

  # every time a 3gram is repeated +1 to total_repetitions  
  finder = TrigramCollocationFinder.from_words(tokens)
  total_repetitions = 0
  for item in finder.ngram_fd.items():
    if item[1] > 1:  # only care about repetition       
      repetitions = item[1]-1 # the first use of a trigram is not a repetition
      total_repetitions += repetitions   

      # just for debugging
      if debug:
        print(item[0])
        print('repeated '+str(repetitions)+' time(s)...')   
        print('found '+str(total_repetitions)+' total repetition(s)...')
  return(total_repetitions, word_count, character_count)


0
191
1036


In [ ]:
#-----------------------------------
# Function to calculate Perplexity
#-----------------------------------
# Stolen from: https://huggingface.co/docs/transformers/perplexity
# when (stride > # of tokens in my_text) or (stride > # of tokens the model can process) we evaluate the model’s perplexity by autoregressively factorizing a sequence and conditioning on the ENTIRE preceding subsequence at each step
# otherwise evaluated with a sliding-window strategy
def calculate_perplexity(my_text, emodel, device, stride=512):
 
  # encode our text
  my_encodings = tokenizer(my_text, return_tensors="pt")

  # not sure if this stride makes any good sense given an output of 250 items
  max_length = emodel.config.n_positions

  nlls = []
  for i in tqdm(range(0, my_encodings.input_ids.size(1), stride)):
      begin_loc = max(i + stride - max_length, 0)
      end_loc = min(i + stride, my_encodings.input_ids.size(1))
      trg_len = end_loc - i  # may be different from stride on last loop
      input_ids = my_encodings.input_ids[:, begin_loc:end_loc].to(device)
      #input_ids = output_encodings.input_ids[:, begin_loc:end_loc]
      target_ids = input_ids.clone()
      target_ids[:, :-trg_len] = -100

      with torch.no_grad():
          outputs = emodel(input_ids, labels=target_ids)
          neg_log_likelihood = outputs[0] * trg_len

      nlls.append(neg_log_likelihood)

  ppl = torch.exp(torch.stack(nlls).sum() / end_loc)
  return(ppl.item())

In [30]:
#------------------------------
# Evaluate all generated text
#------------------------------
# hold evaulations in a dataframe (easily saved/shared as csv)
evaluate_df = pd.DataFrame()

# iterate through all of our generated text
for i, row in generated_text_df.iterrows():
  print(generated_text_df.iloc[i,1],generated_text_df.iloc[i,2],generated_text_df.iloc[i,4])
  total_repetitions, word_count, character_count = calculate_repetitions(generated_text_df.iloc[i,5])
  ppl = calculate_perplexity(generated_text_df.iloc[i,5], emodel, device, 1024)
  evaluate_df = evaluate_df.append(pd.DataFrame({'method':[generated_text_df.iloc[i,1]],
                                                 'prompt_source':[generated_text_df.iloc[i,2]],
                                                 'trial':[generated_text_df.iloc[i,4]],
                                                 'word_count':[word_count], 
                                                 'character_count':[character_count], 
                                                 'total_repetitions':[total_repetitions], 
                                                 'perplexity':[ppl],                                            
                                                 'prompt':[generated_text_df.iloc[i,3]],
                                                 'generated_text':[generated_text_df.iloc[i,5]]}))

# save our results to local CSV after iterating through all prompts
evaluate_df.to_csv('data_evaluation_results.csv') 
files.download('data_evaluation_results.csv')  

Beam Search Tale of Two Cities 1


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [12]:

total_repetitions, word_count, character_count = calculate_repetitions("""It was the best of times, worst of times, it was  HUMAN EVENTFULLY WRONG about half the people.. I could deal with whatever that became, and I want to hear about your lovely post about how This was on SRW... Just like once again, those people that I know say a card cannot be a dupe because they thought about drawing a bad-level card and had the card of an AUTO deck to use.
You'd better think before spam your good, rare warzones, and to review your haxus ults (something their logs don't easily disclose, since their logs are deleted when they lose the battle)because what people keep saying is not possible because they could be receiving a card they really want from SRW...
Annoyed by "admins" how do!!!! you think I got fired "for trying to help players out!" and more annoying, I have been afraid to post or even reply to a modmail or any message sent from SRW because the only admins I've been dealing with today had been bigots and could easily handle banning me, and they even had their friends to help them out.
"SNAPPY -WHAT""", True)
print(total_repetitions)
print(word_count)
print(character_count)

In [ ]:
# old crap
#------------------
# Greedy method
#------------------

# tokenize our text promp
input_ids = tokenizer.encode('I enjoy walking with my cute dog', return_tensors='tf')

# generate text 
gmodel_output = gmodel.generate(input_ids, 
                                num_return_sequences=10, 
                                max_length=250)

# decode the output back into text
output_text = tokenizer.decode(greedy_output[0], skip_special_tokens=True)

# show generated text
print(output_text)

AssertionError: ignored

In [ ]:
#-----------------------
# Calculate Perplexity
#-----------------------
def calculate_perplexity(my_text, emodel, device, stride=512):
  # https://huggingface.co/docs/transformers/perplexity
  # when (stride > # of tokens in my_text) or (stride > # of tokens the model can process) we evaluate the model’s perplexity by autoregressively factorizing a sequence and conditioning on the ENTIRE preceding subsequence at each step
  # otherwise evaluated with a sliding-window strategy

  # encode our text
  my_encodings = tokenizer(my_text, return_tensors="pt")

  # not sure if this stride makes any good sense given an output of 250 items
  max_length = emodel.config.n_positions

  nlls = []
  for i in tqdm(range(0, my_encodings.input_ids.size(1), stride)):
      begin_loc = max(i + stride - max_length, 0)
      end_loc = min(i + stride, my_encodings.input_ids.size(1))
      trg_len = end_loc - i  # may be different from stride on last loop
      input_ids = my_encodings.input_ids[:, begin_loc:end_loc].to(device)
      #input_ids = output_encodings.input_ids[:, begin_loc:end_loc]
      target_ids = input_ids.clone()
      target_ids[:, :-trg_len] = -100

      with torch.no_grad():
          outputs = emodel(input_ids, labels=target_ids)
          neg_log_likelihood = outputs[0] * trg_len

      nlls.append(neg_log_likelihood)

  ppl = torch.exp(torch.stack(nlls).sum() / end_loc)
  return(ppl.item())

In [ ]:
# run ppl function
ppl = calculate_perplexity(output_text, emodel, device, 1024)
ppl

100%|██████████| 1/1 [00:00<00:00, 47.61it/s]


1.3632889986038208

In [ ]:
#------------------------
# Calculate Repetitions
#------------------------
def calculate_repetitions(my_text, debug=False):
  # ignore capitalization and punctuation
  clean_text = my_text.lower().translate(str.maketrans('', '', string.punctuation))

  # word-based tokens (use spaces)
  tokenizer = nltk.RegexpTokenizer(r"\w+")
  tokens = tokenizer.tokenize(clean_text)

  # every time a 3gram is repeated +1 to total_repetitions  
  finder = TrigramCollocationFinder.from_words(tokens)
  total_repetitions = 0
  for item in finder.ngram_fd.items():
    if item[1] > 1:  # only care about repetition       
      repetitions = item[1]-1 # the first use of a trigram is not a repetition
      total_repetitions += repetitions   

      # just for debugging
      if debug:
        print(item[0])
        print('repeated '+str(repetitions)+' time(s)...')   
        print('found '+str(total_repetitions)+' total repetition(s)...')
  return(total_repetitions)

In [ ]:
# run ppl function
rpt = calculate_repetitions("\"Forty-two!\" yelled Loonquawl. \"Is that all you’ve got to show for seven and a half million years’ work?\"\n\n\"I checked it very thoroughly,\" said the computer, \"and that quite definitely is the answer. I think the problem, to be quite honest with you, is that you've never actually known what the question is.\"\n\n\"But it was the Great Question! The Ultimate Question of Life, the Universe and Everything!\" howled Loonquawl.\n\n\"Yes,\" said Deep Thought with the air of one who suffers fools gladly, \"but what actually is it?\"\n\nA slow stupefied silence crept over the men as they stared at the computer and then at each other. \n\n\"Well, you know, it’s just Everything… Everything…\" offered Phouchg weakly.\n\n\"Exactly!\" said Deep Thought. \"So once you do know what the question actually is, you’ll know what the answer means.\"", True)
#rpt = calculate_repetitions("Yes we can change! Yes we can stop!", True)
#rpt = calculate_repetitions(output_text, True)
rpt

('the', 'computer', 'and')
repeated 1 time(s)...
found 1 total repetition(s)...
('what', 'the', 'question')
repeated 1 time(s)...
found 2 total repetition(s)...
('said', 'deep', 'thought')
repeated 1 time(s)...
found 3 total repetition(s)...
('know', 'what', 'the')
repeated 1 time(s)...
found 4 total repetition(s)...


4

In [ ]:
#output_text

output_text.lower().translate(str.maketrans('', '', string.punctuation))

#output_text.lower().replace(string.punctuation,'')
#string.punctuation

'i enjoy walking with my cute dog but im not sure if ill ever be able to walk with my dog im not sure if ill ever be able to walk with my dog\n\nim not sure if ill ever be able to walk with my dog im not sure if ill ever be able to walk with my dog\n\nim not sure if ill ever be able to walk with my dog im not sure if ill ever be able to walk with my dog\n\nim not sure if ill ever be able to walk with my dog im not sure if ill ever be able to walk with my dog\n\nim not sure if ill ever be able to walk with my dog im not sure if ill ever be able to walk with my dog\n\nim not sure if ill ever be able to walk with my dog im not sure if ill ever be able to walk with my dog\n\nim not sure if ill ever be able to walk with my dog im not sure if ill ever be able to walk with my dog\n\nim not'

In [ ]:
# show generated text
print(output_text)

I enjoy walking with my cute dog, but I'm not sure if I'll ever be able to walk with my dog. I'm not sure if I'll ever be able to walk with my dog.

I'm not sure if I'll ever be able to walk with my dog. I'm not sure if I'll ever be able to walk with my dog.

I'm not sure if I'll ever be able to walk with my dog. I'm not sure if I'll ever be able to walk with my dog.

I'm not sure if I'll ever be able to walk with my dog. I'm not sure if I'll ever be able to walk with my dog.

I'm not sure if I'll ever be able to walk with my dog. I'm not sure if I'll ever be able to walk with my dog.

I'm not sure if I'll ever be able to walk with my dog. I'm not sure if I'll ever be able to walk with my dog.

I'm not sure if I'll ever be able to walk with my dog. I'm not sure if I'll ever be able to walk with my dog.

I'm not
